In [2]:
"""
CASCADED FRAUD DETECTION INFERENCE SCRIPT v3.0
===============================================
Production-ready inference using FeatureEngineer for proper 235-feature engineering.

SAVED FILES REQUIRED:
  - {prefix}_stage1.joblib: BNN model, threshold, rule_weights
  - {prefix}_ensemble.joblib: ML models, scalers, rule_specs, feature_names
  - {prefix}_feature_config.json: Feature engineering config
  - {prefix}_user_merchant_stats.joblib: User/merchant statistics

PIPELINE:
  Raw Transaction → FeatureEngineer (235 features) → Scalers → ML Ensemble → Decision
"""

import os
import sys
import json
import joblib
import warnings
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Suppress XGBoost GPU/CPU device mismatch warning (model trained on GPU, inference on CPU)
warnings.filterwarnings('ignore', message='.*Falling back to prediction using DMatrix.*')

# Add ML_Models_4.0 dir to import FeatureEngineer (moved from Fold)
FEATURE_ENGINEER_DIR = r"C:\Users\ADMIN\Documents\TechFiesta_2026\ML_Models_6.0_Latest"
sys.path.insert(0, FEATURE_ENGINEER_DIR)
from feature_engineer import FeatureEngineer

# ============================================================================
# MODEL PATHS
# ============================================================================
MODEL_DIR = r"C:\Users\ADMIN\Documents\TechFiesta_2026\ML_Models_6.0_Latest"
PREFIX = "ultimate_fraud_system_fast.joblib"

STAGE1_FILE = os.path.join(MODEL_DIR, f"{PREFIX}_stage1.joblib")
STAGE2_FILE = os.path.join(MODEL_DIR, f"{PREFIX}_ensemble.joblib")
FEATURE_CONFIG_FILE = os.path.join(MODEL_DIR, f"{PREFIX}_feature_config.json")
USER_STATS_FILE = os.path.join(MODEL_DIR, f"{PREFIX}_user_merchant_stats.joblib")


# ============================================================================
# RULE INTERPRETER (Loads declarative rules from JSON config)
# ============================================================================
# This replaces hardcoded lambdas with a generic interpreter that reads
# rule conditions from the JSON config file saved by trx2.py.
# When rules change in trx2.py, they are automatically picked up here.
# ============================================================================

RULES_CONFIG_FILE = os.path.join(MODEL_DIR, "ultimate_fraud_system_fast.joblib_rules.json")


class RuleInterpreter:
    """
    Generic rule interpreter that evaluates declarative rules from JSON config.
    
    Supports operators:
    - Comparison: >, >=, <, <=, ==, !=
    - Range: between (inclusive)
    - Membership: in
    - Compound: and, or
    - Special: field_ne (compare two fields)
    """
    
    def __init__(self, config_path=None):
        self.rule_weights = {}
        self.rule_conditions = {}
        
        # Try to load from config file
        config_path = config_path or RULES_CONFIG_FILE
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                config = json.load(f)
            self.rule_weights = config.get('rule_weights', {})
            self.rule_conditions = config.get('rule_conditions', {})
            print(f"   ✅ Loaded {len(self.rule_conditions)} rules from config")
        else:
            print(f"   ⚠️ Rules config not found, using fallback")
            self._load_fallback_rules()
        
        # ALWAYS inject new rules (in case config is stale from old training)
        # These will be properly exported after next retrain via trx2.py
        self._inject_new_rules()
    
    def _load_fallback_rules(self):
        """Fallback rules if config file doesn't exist"""
        self.rule_weights = {
            'velocity_attack': 30, 'amount_anomaly_extreme': 35, 'high_amount': 25,
            'night_transaction': 10, 'new_device': 15, 'new_country': 20
        }
        self.rule_conditions = {
            'velocity_attack': {'field': 'txns_last_10min', 'op': 'between', 'value': [5, 10]},
            'amount_anomaly_extreme': {'field': 'amount_vs_user_avg', 'op': '>', 'value': 10},
            'high_amount': {'field': 'amount_vs_user_avg', 'op': '>', 'value': 5},
            'night_transaction': {'field': 'is_night', 'op': '==', 'value': 1},
            'new_device': {'field': 'is_new_device', 'op': '==', 'value': 1},
            'new_country': {'field': 'is_new_country', 'op': '==', 'value': 1},
        }
    
    def _inject_new_rules(self):
        """Inject new rules that may not be in saved config (added after training)"""
        # Repeat Fraud Offender - user with 2+ past frauds
        if 'repeat_fraud_offender' not in self.rule_conditions:
            self.rule_weights['repeat_fraud_offender'] = 85
            self.rule_conditions['repeat_fraud_offender'] = {
                'field': 'user_past_fraud_count', 'op': '>=', 'value': 2
            }
        
        # Micro Transaction Velocity - 3+ micro txns in 5 min (HARD BLOCK - card testing)
        if 'micro_txn_velocity' not in self.rule_conditions:
            self.rule_weights['micro_txn_velocity'] = 85
            self.rule_conditions['micro_txn_velocity'] = {
                'field': 'micro_txn_count_5min', 'op': '>=', 'value': 3
            }
        
        # Update fraud_history_high weight to be higher (was 25, now 65)
        if 'fraud_history_high' in self.rule_weights:
            self.rule_weights['fraud_history_high'] = 65
    
    def _evaluate_condition(self, txn, condition):
        """Evaluate a single condition against a transaction dict"""
        op = condition.get('op', '==')
        
        # Compound conditions (and/or)
        if op == 'and':
            return all(self._evaluate_condition(txn, c) for c in condition.get('conditions', []))
        if op == 'or':
            return any(self._evaluate_condition(txn, c) for c in condition.get('conditions', []))
        
        # Special: compare two fields
        if op == 'field_ne':
            fields = condition.get('value', [])
            if len(fields) == 2:
                v1 = str(txn.get(fields[0], '')).upper()
                v2 = str(txn.get(fields[1], '')).upper()
                return v1 != '' and v2 != '' and v1 != v2
            return False
        
        # Get field value
        field = condition.get('field', '')
        value = txn.get(field, 0)
        target = condition.get('value')
        
        # Handle string comparisons (convert to uppercase for 'in' operator)
        if op == 'in':
            if isinstance(value, str):
                return value.upper() in [str(v).upper() for v in target]
            return value in target
        
        # Numeric comparisons
        try:
            value = float(value) if not isinstance(value, str) else 0
            
            if op == '>': return value > target
            if op == '>=': return value >= target
            if op == '<': return value < target
            if op == '<=': return value <= target
            if op == '==': return value == target
            if op == '!=': return value != target
            if op == 'between':
                low, high = target[0], target[1]
                return low <= value <= high
        except (ValueError, TypeError):
            pass
        
        return False
    
    def evaluate(self, txn_dict):
        """Evaluate all rules, return max score and triggered rules"""
        triggered = []
        max_score = 0
        
        for rule_name, condition in self.rule_conditions.items():
            try:
                if self._evaluate_condition(txn_dict, condition):
                    weight = self.rule_weights.get(rule_name, 0)
                    triggered.append((rule_name, weight))
                    max_score = max(max_score, weight)
            except Exception:
                pass
        
        return max_score, triggered
    
    def get_rule_signals(self, txn_dict):
        """Get binary signals for all rules (for BNN input)"""
        signals = []
        for rule_name in self.rule_conditions.keys():
            try:
                condition = self.rule_conditions[rule_name]
                signals.append(1.0 if self._evaluate_condition(txn_dict, condition) else 0.0)
            except:
                signals.append(0.0)
        return signals
    
    def get_rule_signals_ordered(self, txn_dict, rule_names):
        """Get binary signals in EXACT order specified by rule_names.
        
        This is used for BNN inference to ensure signals match training order.
        """
        signals = []
        for rule_name in rule_names:
            try:
                condition = self.rule_conditions.get(rule_name, {})
                signals.append(1.0 if self._evaluate_condition(txn_dict, condition) else 0.0)
            except:
                signals.append(0.0)
        return signals


# Legacy alias for compatibility
class RuleEngine(RuleInterpreter):
    """Alias for backward compatibility"""
    def __init__(self, rule_specs=None, config_path=None):
        super().__init__(config_path)
        # If rule_specs provided (old format), merge with config
        if rule_specs:
            for rule_name, weight in rule_specs:
                self.rule_weights[rule_name] = weight



# ============================================================================
# BNN INFERENCE
# ============================================================================
def predict_bnn(txn_dict, rule_engine, bnn_model, bnn_threshold, bnn_rule_names=None):
    """Predict using BNN with rule signals from RuleInterpreter
    
    If bnn_rule_names is provided, signals will be generated in that exact order
    to match the order used during BNN training.
    """
    if bnn_model is None:
        return {'is_high_risk': False, 'probability': 0.0}
    try:
        # Get rule signals - use saved order if available, otherwise use dict order
        if bnn_rule_names:
            signals = np.array([rule_engine.get_rule_signals_ordered(txn_dict, bnn_rule_names)]).reshape(1, -1)
        else:
            signals = np.array([rule_engine.get_rule_signals(txn_dict)]).reshape(1, -1)
        prob = bnn_model.predict_proba(signals)[0, 1]
        return {'is_high_risk': prob >= bnn_threshold, 'probability': float(prob)}
    except Exception as e:
        return {'is_high_risk': False, 'probability': 0.0}


# ============================================================================
# LOAD SYSTEM
# ============================================================================
def load_system():
    print("=" * 70)
    print("🔮 LOADING CASCADED FRAUD DETECTION SYSTEM")
    print("=" * 70)
    
    system = {}
    
    # ===== Stage 1: BNN =====
    print("\n📦 Loading Stage 1 (Rules + BNN)...")
    if os.path.exists(STAGE1_FILE):
        stage1 = joblib.load(STAGE1_FILE)
        system['bnn_model'] = stage1.get('bnn_model')
        system['bnn_threshold'] = stage1.get('bnn_threshold', 0.5)
        system['hard_block_threshold'] = stage1.get('hard_block_threshold', 85) / 100.0
        system['bnn_rule_names'] = stage1.get('bnn_rule_names', [])  # Rule order for BNN
        print(f"   ✅ BNN loaded (threshold={system['bnn_threshold']:.3f})")
        if system['bnn_rule_names']:
            print(f"   ✅ BNN rule order: {len(system['bnn_rule_names'])} rules")
    else:
        print(f"   ⚠️ Stage 1 not found")
        system['bnn_model'] = None
        system['bnn_threshold'] = 0.5
        system['hard_block_threshold'] = 0.85
    
    # ===== Stage 2: ML Ensemble + Scalers =====
    print("\n📦 Loading Stage 2 (ML Ensemble + Scalers)...")
    if os.path.exists(STAGE2_FILE):
        stage2 = joblib.load(STAGE2_FILE)
        system['models'] = stage2.get('models', {})
        system['optimal_weights'] = stage2.get('optimal_weights', {})
        system['optimal_threshold'] = stage2.get('optimal_threshold', 0.390)
        system['scaler'] = stage2.get('scaler')
        system['quantile_transformer'] = stage2.get('quantile_transformer')
        system['feature_names'] = stage2.get('feature_names', [])
        system['rule_specs'] = stage2.get('rule_specs', [])
        print(f"   ✅ {len(system['models'])} models, {len(system['feature_names'])} features")
        print(f"   ✅ Scalers: RobustScaler + QuantileTransformer")
    else:
        print(f"   ❌ Stage 2 not found")
        return None
    
    # ===== Feature Engineer =====
    print("\n📦 Loading Feature Engineering Config...")
    if os.path.exists(FEATURE_CONFIG_FILE) and os.path.exists(USER_STATS_FILE):
        system['feature_engineer'] = FeatureEngineer(FEATURE_CONFIG_FILE, USER_STATS_FILE)
        print(f"   ✅ FeatureEngineer loaded")
    else:
        print(f"   ⚠️ Feature config not found - using basic features")
        system['feature_engineer'] = None
    
    # Rule Engine - Now loads from config file automatically!
    print("\n📦 Loading Rule Engine from config...")
    system['rule_engine'] = RuleEngine(config_path=RULES_CONFIG_FILE)
    print(f"   ✅ Rule Engine ({len(system['rule_engine'].rule_conditions)} rules)")
    
    return system


# ============================================================================
# INFERENCE
# ============================================================================
def predict(raw_transaction, system, verbose=True):
    """
    Full cascaded prediction with proper feature engineering.
    
    raw_transaction: dict with raw transaction fields (amount, timestamp, user_id, etc.)
    verbose: if True, print step-by-step pipeline progress
    """
    result = {'raw_input': raw_transaction}
    
    # ===== STEP 1: Feature Engineering =====
    if verbose:
        print(f"\n   📥 STEP 1: Raw Input")
        print(f"      └─ Received {len(raw_transaction)} raw columns")
        print(f"      └─ Amount: ${raw_transaction.get('amount', 0):,.2f}")
        print(f"      └─ User: {raw_transaction.get('user_id', 'N/A')}")
    
    if system['feature_engineer']:
        # Create DataFrame with required columns
        df = pd.DataFrame([raw_transaction])
        df_engineered, feature_cols = system['feature_engineer'].engineer_features(df)
        full_features = df_engineered[system['feature_names']].iloc[0].to_dict()
        
        # IMPORTANT: Merge raw transaction fields for rule engine
        # Some rule fields (txns_last_10min, is_burst_txn) may not be in training features
        # but need to be accessible by the rule engine
        for key, value in raw_transaction.items():
            if key not in full_features:
                full_features[key] = value
        
        if verbose:
            print(f"\n   🔧 STEP 2: Feature Engineering")
            print(f"      └─ Input columns: {len(raw_transaction)}")
            print(f"      └─ Engineered features: {len(system['feature_names'])}")
            print(f"      └─ Config: feature_config.json ✓")
            print(f"      └─ User/Merchant stats loaded ✓")
    else:
        # Fallback: fill missing with 0
        full_features = {feat: raw_transaction.get(feat, 0) for feat in system['feature_names']}
        if verbose:
            print(f"\n   🔧 STEP 2: Feature Engineering (fallback mode)")
            print(f"      └─ Using raw features with defaults")
    
    # ===== STEP 2: Rule Engine =====
    if system['rule_engine']:
        rule_score, triggered = system['rule_engine'].evaluate(full_features)
        rule_score_norm = rule_score / 100.0
        result['rule_score'] = rule_score
        result['triggered_rules'] = [r[0] for r in triggered]
        
        if verbose:
            print(f"\n   🚨 STEP 3: Rule Engine (Stage 1)")
            print(f"      └─ Rules evaluated: {len(system['rule_engine'].rule_conditions)}")
            print(f"      └─ Rules triggered: {len(triggered)}")
            print(f"      └─ Score: {rule_score}% (threshold: {int(system['hard_block_threshold']*100)}%)")
            if triggered:
                print(f"      └─ Matched: {', '.join([r[0] for r in triggered[:3]])}")
        
        if rule_score_norm >= system['hard_block_threshold']:
            result['decision'] = 'FRAUD (HARD BLOCK)'
            result['final_score'] = 1.0
            if verbose:
                print(f"\n   🛑 HARD BLOCK TRIGGERED - Skipping ML")
            return result
    else:
        rule_score, rule_score_norm = 0, 0
        result['rule_score'] = 0
        result['triggered_rules'] = []
    
    # ===== STEP 3: BNN High-Risk =====
    if system['bnn_model'] and system['rule_engine']:
        bnn_result = predict_bnn(full_features, system['rule_engine'],
                                  system['bnn_model'], system['bnn_threshold'],
                                  system.get('bnn_rule_names'))
        is_high_risk = bnn_result['is_high_risk']
        result['bnn_probability'] = bnn_result['probability']
        
        if verbose:
            print(f"\n   🧠 STEP 4: BNN High-Risk Identifier")
            print(f"      └─ Rule signals extracted: {len(system['rule_engine'].rule_conditions)}")
            print(f"      └─ Probability: {bnn_result['probability']:.3f}")
            print(f"      └─ Threshold: {system['bnn_threshold']:.3f}")
            print(f"      └─ Classification: {'HIGH RISK ⚠️' if is_high_risk else 'NORMAL ✓'}")
    else:
        is_high_risk = rule_score_norm >= 0.60
    result['is_high_risk'] = is_high_risk
    
    # ===== STEP 4: Scaling =====
    X = pd.DataFrame([[full_features[f] for f in system['feature_names']]], 
                      columns=system['feature_names'])
    
    if verbose:
        print(f"\n   📏 STEP 5: Hybrid Scaling")
    
    # Apply scalers
    if system['scaler']:
        X_scaled = system['scaler'].transform(X)
        if verbose:
            print(f"      └─ RobustScaler applied ✓")
        if system['quantile_transformer']:
            X_scaled = system['quantile_transformer'].transform(X_scaled)
            if verbose:
                print(f"      └─ QuantileTransformer applied ✓")
    else:
        X_scaled = X.values
        if verbose:
            print(f"      └─ No scalers (raw features)")
    
    if verbose:
        print(f"      └─ Output shape: {X_scaled.shape}")
    
    # ===== STEP 5: ML Ensemble =====
    if verbose:
        print(f"\n   🤖 STEP 6: ML Ensemble Prediction")
    
    ml_pred = 0.0
    model_preds = {}
    for name, model in system['models'].items():
        weight = system['optimal_weights'].get(name, 1/len(system['models']))
        try:
            pred = model.predict_proba(X_scaled)[0, 1]
            model_preds[name] = pred
            ml_pred += weight * pred
            if verbose:
                print(f"      └─ {name}: {pred*100:.1f}% × {weight*100:.1f}% = {weight * pred * 100:.1f}%")
        except Exception as e:
            if verbose:
                print(f"      └─ {name}: ERROR - {e}")
    
    result['ml_prediction'] = ml_pred
    result['model_predictions'] = model_preds
    
    if verbose:
        print(f"      └─ ML Fraud Probability: {ml_pred*100:.1f}%")
    
    # ===== STEP 6: Blending =====
    # FIX: Use HIGH RISK blend if EITHER:
    #   1. BNN classifies as HIGH RISK, OR
    #   2. Rule score >= 60% (gray zone with strong rule signals)
    # This ensures strong rule signals (like velocity attacks) are not diluted by BNN
    use_high_risk_blend = is_high_risk or rule_score >= 60  # rule_score is 0-100 scale
    
    if use_high_risk_blend:
        final_score = ml_pred * 0.65 + rule_score_norm * 0.35
        if is_high_risk and rule_score >= 60:
            result['blend_type'] = 'High Risk (BNN + Rules)'
        elif is_high_risk:
            result['blend_type'] = 'High Risk (BNN)'
        else:
            result['blend_type'] = 'High Risk (Rules >= 60%)'
        blend_formula = f"({ml_pred*100:.1f}% × 65%) + ({rule_score_norm*100:.1f}% × 35%)"
    else:
        final_score = ml_pred * 0.90 + rule_score_norm * 0.10
        result['blend_type'] = 'Low Risk'
        blend_formula = f"({ml_pred*100:.1f}% × 90%) + ({rule_score_norm*100:.1f}% × 10%)"
    
    result['final_score'] = final_score
    result['threshold'] = system['optimal_threshold']
    result['decision'] = 'FRAUD' if final_score >= system['optimal_threshold'] else 'LEGITIMATE'
    
    # Determine risk level for interpretability
    # Risk Level Thresholds:
    # - LOW: 0-25% (safe, minimal monitoring)
    # - MEDIUM: 25-36% (standard monitoring, may review)
    # - HIGH: 36%+ (fraud detected, block)
    if final_score >= 0.36:
        risk_level = "HIGH"
    elif final_score >= 0.25:
        risk_level = "MEDIUM"
    else:
        risk_level = "LOW"
    result['risk_level'] = risk_level
    
    if verbose:
        print(f"\n   ⚡ STEP 7: Cascaded Blending")
        print(f"      └─ Mode: {result['blend_type']} (BNN classification)")
        print(f"      └─ Formula: {blend_formula}")
        print(f"      └─ Final Fraud Probability: {final_score*100:.1f}%")
    
    if verbose:
        # Create visual progress bar
        bar_length = 20
        filled = int(final_score * bar_length)
        bar = "█" * filled + "░" * (bar_length - filled)
        
        threshold_pct = system['optimal_threshold'] * 100
        final_pct = final_score * 100
        
        print(f"\n   🎯 FINAL DECISION")
        print(f"      ┌{'─' * 40}┐")
        print(f"      │ Fraud Probability: {final_pct:5.1f}%           │")
        print(f"      │ Decision Threshold: {threshold_pct:5.1f}%          │")
        print(f"      │ Risk Level: {risk_level:8}               │")
        print(f"      │ [{bar}] {final_pct:.0f}%     │")
        print(f"      └{'─' * 40}┘")
        
        if result['decision'] == 'FRAUD' or result['decision'] == 'FRAUD (HARD BLOCK)':
            print(f"\n      🚨 FRAUD DETECTED....TRANSACTION BLOCKED")
        elif risk_level == 'MEDIUM':
            print(f"\n      ⚠️  ELEVATED RISK DETECTED")
            print(f"         Transaction has been flagged for additional verification.")
            print(f"         Alert dispatched to Manual Review Team for final decision.")
            result['manual_review'] = True
        else:
            print(f"\n      ✅ VERDICT: LEGITIMATE ({final_pct:.1f}% < {threshold_pct:.1f}%)")
    
    return result


# ============================================================================
# TEST CASES - Complete 37-column dataset schema
# Matches RealisticFraudDatasetGenerator columns exactly
# ============================================================================

# Current timestamp for test transactions
NOW = datetime.now()

TEST_CASES = [
    # ===== TEST 1: OBVIOUS FRAUD - Stolen Card from High-Risk Country =====
    {"name": "🚨 STOLEN CARD", 
     "description": "High amount from new device, high-risk country (NG)",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_001",
         "user_id": "USER_SUSPECT_001",
         "timestamp": NOW,
         "amount": 2500,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "NG",  # Nigeria - high risk
         "billing_city": "Lagos",
         "shipping_country": "US",
         "shipping_city": "New York",
         "latitude": 6.5244,
         "longitude": 3.3792,
         # === Device Fields ===
         "ip_address": "197.210.45.123",
         "device_location_lat": 6.5244,
         "device_location_lon": 3.3792,
         "device_id": "DEV_NEW_001",
         "device_type": "mobile",
         "device_os": "Android",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Linux; Android 11)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_GIFT_001",
         "merchant_name": "GiftCardsNG",
         "merchant_category": "Gift Cards",
         "merchant_country": "NG",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=5),  # New user
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "pending",
         "user_risk_flag": 1,
         "user_total_txn_count": 2,
         "user_total_txn_amount": 100,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.5,
         "failed_login_attempts_last_24h": 3,
     }},
    
    # ===== TEST 2: VELOCITY ATTACK - Multiple rapid transactions =====
    {"name": "🚨 VELOCITY ATTACK", 
     "description": "Rapid transactions in burst (5 in 10 min)",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_002",
         "user_id": "USER_VELOCITY_001",
         "timestamp": NOW,
         "amount": 450,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "US",
         "billing_city": "Miami",
         "shipping_country": "US",
         "shipping_city": "Miami",
         "latitude": 25.7617,
         "longitude": -80.1918,
         # === Device Fields ===
         "ip_address": "72.45.123.89",
         "device_location_lat": 25.7617,
         "device_location_lon": -80.1918,
         "device_id": "DEV_BURST_001",
         "device_type": "desktop",
         "device_os": "Windows",
         "browser": "Firefox",
         "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_CRYPTO_001",
         "merchant_name": "CryptoExchange",
         "merchant_category": "Crypto",
         "merchant_country": "US",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=90),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 50,
         "user_total_txn_amount": 5000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.003,  # ~4 minutes ago (velocity attack)
         "failed_login_attempts_last_24h": 0,
         # VELOCITY ATTACK indicators - explicit fields for rule engine
         "txns_last_10min": 6,  # 6 transactions in last 10 minutes (triggers velocity_attack rule)
         "is_burst_txn": 1,     # Mark as burst transaction
     }},
    
    # ===== TEST 3: CARD TESTING PATTERN - Micro txns followed by large =====
    {"name": "🚨 CARD TESTING", 
     "description": "Small $3 test, now $800 purchase",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_003",
         "user_id": "USER_CARDTEST_001",
         "timestamp": NOW,
         "amount": 800,  # Large amount after testing
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "US",
         "billing_city": "Chicago",
         "shipping_country": "US",
         "shipping_city": "Chicago",
         "latitude": 41.8781,
         "longitude": -87.6298,
         # === Device Fields ===
         "ip_address": "98.210.55.42",
         "device_location_lat": 41.8781,
         "device_location_lon": -87.6298,
         "device_id": "DEV_TEST_001",
         "device_type": "mobile",
         "device_os": "iOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_TECH_001",
         "merchant_name": "TechGadgets",
         "merchant_category": "Technology",
         "merchant_country": "US",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=3),  # Very new account
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "partial",
         "user_risk_flag": 0,
         "user_total_txn_count": 3,  # Only 3 previous micro transactions
         "user_total_txn_amount": 9,  # $3 + $3 + $3 = $9 (micro amounts)
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.01,
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 4: SUSPICIOUS - Night + Failed KYC + Foreign Merchant =====
    {"name": "⚠️ SUSPICIOUS", 
     "description": "2AM txn, failed KYC, geo mismatch",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_004",
         "user_id": "USER_SUSPECT_002",
         "timestamp": datetime(2024, 12, 20, 2, 30, 0),  # 2:30 AM
         "amount": 600,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "RU",  # Russia - mismatch
         "billing_city": "Moscow",
         "shipping_country": "RU",
         "shipping_city": "Moscow",
         "latitude": 55.7558,
         "longitude": 37.6173,
         # === Device Fields ===
         "ip_address": "95.84.192.45",
         "device_location_lat": 55.7558,
         "device_location_lon": 37.6173,
         "device_id": "DEV_RU_001",
         "device_type": "desktop",
         "device_os": "Windows",
         "browser": "Edge",
         "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Edge/119.0",
         # === Merchant Fields ===
         "merchant_id": "MERCH_TRANSFER_001",
         "merchant_name": "MoneyTransferRU",
         "merchant_category": "Money Transfer",
         "merchant_country": "RU",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=60),
         "user_country": "US",  # User is from US
         "user_segment": "retail",
         "kyc_status": "failed",  # Failed KYC
         "user_risk_flag": 1,
         "user_total_txn_count": 10,
         "user_total_txn_amount": 1500,
         "user_past_fraud_count": 1,  # Has fraud history
         "days_since_last_txn": 5,
         "failed_login_attempts_last_24h": 4,
     }},
    
    # ===== TEST 5: NORMAL - Trusted user, regular purchase =====
    {"name": "✅ NORMAL", 
     "description": "Established user, small daytime purchase",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_005",
         "user_id": "USER_TRUSTED_001",
         "timestamp": datetime(2024, 12, 20, 14, 30, 0),  # 2:30 PM
         "amount": 45,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "POS",
         # === Location Fields ===
         "billing_country": "US",
         "billing_city": "San Francisco",
         "shipping_country": "US",
         "shipping_city": "San Francisco",
         "latitude": 37.7749,
         "longitude": -122.4194,
         # === Device Fields ===
         "ip_address": "50.193.45.78",
         "device_location_lat": 37.7749,
         "device_location_lon": -122.4194,
         "device_id": "DEV_TRUSTED_001",
         "device_type": "mobile",
         "device_os": "iOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_FOOD_001",
         "merchant_name": "LocalCafe",
         "merchant_category": "Food",
         "merchant_country": "US",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=730),  # 2 years old
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 500,
         "user_total_txn_amount": 25000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 2,
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 6: NORMAL - High-value trusted customer =====
    {"name": "✅ TRUSTED HIGH", 
     "description": "Luxury spender, consistent pattern",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_006",
         "user_id": "USER_VIP_001",
         "timestamp": datetime(2024, 12, 20, 11, 0, 0),  # 11 AM
         "amount": 1200,  # High but within user's normal range
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "US",
         "billing_city": "Beverly Hills",
         "shipping_country": "US",
         "shipping_city": "Beverly Hills",
         "latitude": 34.0736,
         "longitude": -118.4004,
         # === Device Fields ===
         "ip_address": "76.127.89.45",
         "device_location_lat": 34.0736,
         "device_location_lon": -118.4004,
         "device_id": "DEV_VIP_001",
         "device_type": "desktop",
         "device_os": "macOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_0)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_TRAVEL_001",
         "merchant_name": "LuxuryTravel",
         "merchant_category": "Travel",
         "merchant_country": "US",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=1095),  # 3 years old
         "user_country": "US",
         "user_segment": "corporate",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 1200,
         "user_total_txn_amount": 480000,  # Avg $400/txn - luxury spender
         "user_past_fraud_count": 0,
         "days_since_last_txn": 3,
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 7: IMPOSSIBLE TRAVEL - User in 2 locations too fast =====
    {"name": "🚨 IMPOSSIBLE TRAVEL", 
     "description": "NYC 2 hours ago, now in London (speed > 900 km/h)",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_007",
         "user_id": "USER_TRAVEL_001",
         "timestamp": NOW,
         "amount": 1500,
         "currency": "GBP",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "GB",
         "billing_city": "London",
         "shipping_country": "GB",
         "shipping_city": "London",
         "latitude": 51.5074,
         "longitude": -0.1278,
         # === Device Fields ===
         "ip_address": "82.132.45.67",
         "device_location_lat": 51.5074,
         "device_location_lon": -0.1278,
         "device_id": "DEV_TRAVEL_001",
         "device_type": "mobile",
         "device_os": "Android",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Linux; Android 13)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_RETAIL_UK",
         "merchant_name": "Harrods",
         "merchant_category": "Retail",
         "merchant_country": "GB",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=365),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 100,
         "user_total_txn_amount": 15000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.08,  # ~2 hours - impossible travel from NYC
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 8: ACCOUNT TAKEOVER - New device + failed logins + night =====
    {"name": "🚨 ACCOUNT TAKEOVER", 
     "description": "Multiple failed logins, new device, 3AM purchase",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_008",
         "user_id": "USER_ATO_001",
         "timestamp": datetime(2024, 12, 20, 3, 15, 0),  # 3:15 AM
         "amount": 2000,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "BR",
         "billing_city": "São Paulo",
         "shipping_country": "BR",
         "shipping_city": "São Paulo",
         "latitude": -23.5505,
         "longitude": -46.6333,
         # === Device Fields ===
         "ip_address": "177.92.45.123",
         "device_location_lat": -23.5505,
         "device_location_lon": -46.6333,
         "device_id": "DEV_NEW_ATO_001",
         "device_type": "desktop",
         "device_os": "Linux",
         "browser": "Brave",
         "user_agent": "Mozilla/5.0 (X11; Linux x86_64)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_ELECTRONICS",
         "merchant_name": "ElectronicsBR",
         "merchant_category": "Technology",
         "merchant_country": "BR",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=500),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 200,
         "user_total_txn_amount": 20000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 15,
         "failed_login_attempts_last_24h": 5,  # Multiple failed logins
     }},
    
    # ===== TEST 9: DEVICE FINGERPRINT CHAOS - Multiple devices/IPs =====
    {"name": "🚨 DEVICE CHAOS", 
     "description": "5+ devices and 5+ IPs in 24 hours",
     "transaction": {
         "transaction_id": "TXN_TEST_009",
         "user_id": "USER_CHAOS_001",
         "timestamp": NOW,
         "amount": 750,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "US",
         "billing_city": "Denver",
         "shipping_country": "US",
         "shipping_city": "Denver",
         "latitude": 39.7392,
         "longitude": -104.9903,
         "ip_address": "192.168.50.99",
         "device_location_lat": 39.7392,
         "device_location_lon": -104.9903,
         "device_id": "DEV_CHAOS_009",
         "device_type": "tablet",
         "device_os": "Android",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Linux; Android 12)",
         "merchant_id": "MERCH_GAMING_001",
         "merchant_name": "GameStore",
         "merchant_category": "Gaming",
         "merchant_country": "US",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=30),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 20,
         "user_total_txn_amount": 2000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 1,
         "failed_login_attempts_last_24h": 0,
         # Device chaos indicators
         "distinct_devices_24hr": 7,
         "distinct_ips_24hr": 6,
     }},
    
    # ===== TEST 10: NEW COUNTRY HIGH AMOUNT =====
    {"name": "🚨 NEW COUNTRY HIGH", 
     "description": "First transaction from new country, high amount",
     "transaction": {
         "transaction_id": "TXN_TEST_010",
         "user_id": "USER_NEWCOUNTRY_001",
         "timestamp": NOW,
         "amount": 1800,
         "currency": "EUR",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "DE",
         "billing_city": "Berlin",
         "shipping_country": "DE",
         "shipping_city": "Berlin",
         "latitude": 52.5200,
         "longitude": 13.4050,
         "ip_address": "91.64.123.45",
         "device_location_lat": 52.5200,
         "device_location_lon": 13.4050,
         "device_id": "DEV_NEWCOUNTRY_001",
         "device_type": "mobile",
         "device_os": "iOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0)",
         "merchant_id": "MERCH_LUXURY_DE",
         "merchant_name": "LuxuryBerlin",
         "merchant_category": "Luxury",
         "merchant_country": "DE",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=200),
         "user_country": "US",  # User is from US, billing now from DE
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 80,
         "user_total_txn_amount": 8000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 3,
         "failed_login_attempts_last_24h": 0,
         # New country indicator
         "is_new_country": 1,
     }},
    
    # ===== TEST 11: REPEAT FRAUD OFFENDER =====
    {"name": "🚨 REPEAT OFFENDER", 
     "description": "User with past fraud history making high-value purchase",
     "transaction": {
         "transaction_id": "TXN_TEST_011",
         "user_id": "USER_REPEAT_001",
         "timestamp": NOW,
         "amount": 500,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "US",
         "billing_city": "Atlanta",
         "shipping_country": "US",
         "shipping_city": "Atlanta",
         "latitude": 33.7490,
         "longitude": -84.3880,
         "ip_address": "68.45.123.78",
         "device_location_lat": 33.7490,
         "device_location_lon": -84.3880,
         "device_id": "DEV_REPEAT_001",
         "device_type": "desktop",
         "device_os": "Windows",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
         "merchant_id": "MERCH_ELEC_001",
         "merchant_name": "ElectronicsUSA",
         "merchant_category": "Electronics",
         "merchant_country": "US",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=400),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 1,
         "user_total_txn_count": 150,
         "user_total_txn_amount": 15000,
         "user_past_fraud_count": 3,  # Has 3 past frauds!
         "days_since_last_txn": 7,
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 12: MICRO TRANSACTION PATTERN =====
    {"name": "⚠️ MICRO TXN PATTERN", 
     "description": "Multiple $1-$5 transactions in quick succession",
     "transaction": {
         "transaction_id": "TXN_TEST_012",
         "user_id": "USER_MICRO_001",
         "timestamp": NOW,
         "amount": 2.50,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "US",
         "billing_city": "Portland",
         "shipping_country": "US",
         "shipping_city": "Portland",
         "latitude": 45.5152,
         "longitude": -122.6784,
         "ip_address": "73.42.156.89",
         "device_location_lat": 45.5152,
         "device_location_lon": -122.6784,
         "device_id": "DEV_MICRO_001",
         "device_type": "mobile",
         "device_os": "Android",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Linux; Android 13)",
         "merchant_id": "MERCH_DONATION_001",
         "merchant_name": "CharityDonate",
         "merchant_category": "Nonprofit",
         "merchant_country": "US",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=10),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "pending",
         "user_risk_flag": 0,
         "user_total_txn_count": 8,
         "user_total_txn_amount": 24,  # 8 transactions of $3 average
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.001,  # Very recent
         "failed_login_attempts_last_24h": 0,
         # Micro transaction indicators
         "micro_txn_count_5min": 4,
         "time_since_last_micro_txn_min": 1,
     }},
    
    # ===== TEST 13: SPEED OF LIGHT VIOLATION =====
    {"name": "🚨 TELEPORTATION", 
     "description": "Physically impossible travel (>1500 km/h)",
     "transaction": {
         "transaction_id": "TXN_TEST_013",
         "user_id": "USER_TELEPORT_001",
         "timestamp": NOW,
         "amount": 900,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "JP",
         "billing_city": "Tokyo",
         "shipping_country": "JP",
         "shipping_city": "Tokyo",
         "latitude": 35.6762,
         "longitude": 139.6503,
         "ip_address": "103.5.140.50",
         "device_location_lat": 35.6762,
         "device_location_lon": 139.6503,
         "device_id": "DEV_TELEPORT_001",
         "device_type": "mobile",
         "device_os": "iOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0)",
         "merchant_id": "MERCH_RETAIL_JP",
         "merchant_name": "TokyoShop",
         "merchant_category": "Retail",
         "merchant_country": "JP",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=180),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 60,
         "user_total_txn_amount": 6000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.02,  # ~30 min ago - was in NYC
         "failed_login_attempts_last_24h": 0,
         # Speed of light violation
         "travel_speed_kmh": 2000,  # NYC to Tokyo in 30 min = impossible!
     }},
    
    # ===== TEST 14: GRAY ZONE - Borderline Case =====
    {"name": "⚠️ GRAY ZONE", 
     "description": "Multiple weak signals, borderline decision",
     "transaction": {
         "transaction_id": "TXN_TEST_014",
         "user_id": "USER_GRAY_001",
         "timestamp": datetime(2024, 12, 20, 22, 45, 0),  # Late night 10:45 PM
         "amount": 350,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "US",
         "billing_city": "Seattle",
         "shipping_country": "US",
         "shipping_city": "Seattle",
         "latitude": 47.6062,
         "longitude": -122.3321,
         "ip_address": "67.82.45.123",
         "device_location_lat": 47.6062,
         "device_location_lon": -122.3321,
         "device_id": "DEV_GRAY_001",
         "device_type": "desktop",
         "device_os": "macOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_0)",
         "merchant_id": "MERCH_ONLINE_001",
         "merchant_name": "OnlineStore",
         "merchant_category": "Retail",
         "merchant_country": "US",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=45),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 15,
         "user_total_txn_amount": 800,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 4,
         "failed_login_attempts_last_24h": 1,
         # Weak signals
         "is_night": 1,
         "is_new_device": 1,
     }},
    
    # ===== TEST 15: CORPORATE HIGH-VALUE LEGITIMATE =====
    {"name": "✅ CORPORATE LEGIT", 
     "description": "Large corporate purchase, established account",
     "transaction": {
         "transaction_id": "TXN_TEST_015",
         "user_id": "USER_CORP_001",
         "timestamp": datetime(2024, 12, 20, 10, 0, 0),  # 10 AM business hours
         "amount": 15000,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "B2B",
         "billing_country": "US",
         "billing_city": "New York",
         "shipping_country": "US",
         "shipping_city": "New York",
         "latitude": 40.7128,
         "longitude": -74.0060,
         "ip_address": "64.233.160.0",  # Corporate IP
         "device_location_lat": 40.7128,
         "device_location_lon": -74.0060,
         "device_id": "DEV_CORP_001",
         "device_type": "desktop",
         "device_os": "Windows",
         "browser": "Edge",
         "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Edge/119.0",
         "merchant_id": "MERCH_B2B_001",
         "merchant_name": "B2BSupplies",
         "merchant_category": "B2B",
         "merchant_country": "US",
         "payment_method": "wire",
         "user_signup_date": NOW - timedelta(days=1500),  # 4+ years
         "user_country": "US",
         "user_segment": "corporate",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 2000,
         "user_total_txn_amount": 5000000,  # $5M historical
         "user_past_fraud_count": 0,
         "days_since_last_txn": 1,
         "failed_login_attempts_last_24h": 0,
     }},
]


# ============================================================================
# MAIN
# ============================================================================
def main():
    system = load_system()
    if not system:
        print("\n❌ Failed to load system")
        return
    
    print("\n" + "=" * 70)
    print("🧪 EVALUATING TEST CASES")
    print("=" * 70)
    
    for i, tc in enumerate(TEST_CASES, 1):
        print(f"\n{'━' * 70}")
        print(f"📋 TEST {i}/{len(TEST_CASES)}: {tc['name']}")
        print(f"   {tc['description']}")
        print(f"{'━' * 70}")
        
        # Predict with verbose=True shows all pipeline steps
        result = predict(tc['transaction'], system, verbose=True)
    
    print("\n" + "=" * 70)
    print("✅ ALL TESTS COMPLETE")
    print("=" * 70)


if __name__ == "__main__":
    main()


🔮 LOADING CASCADED FRAUD DETECTION SYSTEM

📦 Loading Stage 1 (Rules + BNN)...
   ✅ BNN loaded (threshold=0.491)
   ✅ BNN rule order: 27 rules

📦 Loading Stage 2 (ML Ensemble + Scalers)...
   ✅ 3 models, 235 features
   ✅ Scalers: RobustScaler + QuantileTransformer

📦 Loading Feature Engineering Config...
   📊 Total training features: 235
✅ Feature config loaded: C:\Users\ADMIN\Documents\TechFiesta_2026\ML_Models_6.0_Latest\ultimate_fraud_system_fast.joblib_feature_config.json
   👤 Users in stats: 8000
   🏪 Merchants in stats: 2000
✅ User/Merchant stats loaded: C:\Users\ADMIN\Documents\TechFiesta_2026\ML_Models_6.0_Latest\ultimate_fraud_system_fast.joblib_user_merchant_stats.joblib
   ✅ FeatureEngineer loaded

📦 Loading Rule Engine from config...
   ✅ Loaded 29 rules from config
   ✅ Rule Engine (29 rules)

🧪 EVALUATING TEST CASES

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📋 TEST 1/15: 🚨 STOLEN CARD
   High amount from new device, high-risk country (NG)
━━━━

In [1]:
"""
CASCADED FRAUD DETECTION INFERENCE SCRIPT v3.0
===============================================
Production-ready inference using FeatureEngineer for proper 235-feature engineering.

SAVED FILES REQUIRED:
  - {prefix}_stage1.joblib: BNN model, threshold, rule_weights
  - {prefix}_ensemble.joblib: ML models, scalers, rule_specs, feature_names
  - {prefix}_feature_config.json: Feature engineering config
  - {prefix}_user_merchant_stats.joblib: User/merchant statistics

PIPELINE:
  Raw Transaction → FeatureEngineer (235 features) → Scalers → ML Ensemble → Decision
"""

import os
import sys
import json
import joblib
import warnings
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Suppress XGBoost GPU/CPU device mismatch warning (model trained on GPU, inference on CPU)
warnings.filterwarnings('ignore', message='.*Falling back to prediction using DMatrix.*')

# Add ML_Models_4.0 dir to import FeatureEngineer (moved from Fold)
FEATURE_ENGINEER_DIR = r"C:\Users\ADMIN\Documents\TechFiesta_2026\ML_Models_6.0_Latest"
sys.path.insert(0, FEATURE_ENGINEER_DIR)
from feature_engineer import FeatureEngineer

# ============================================================================
# MODEL PATHS
# ============================================================================
MODEL_DIR = r"C:\Users\ADMIN\Documents\TechFiesta_2026\ML_Models_6.0_Latest"
PREFIX = "ultimate_fraud_system_fast.joblib"

STAGE1_FILE = os.path.join(MODEL_DIR, f"{PREFIX}_stage1.joblib")
STAGE2_FILE = os.path.join(MODEL_DIR, f"{PREFIX}_ensemble.joblib")
FEATURE_CONFIG_FILE = os.path.join(MODEL_DIR, f"{PREFIX}_feature_config.json")
USER_STATS_FILE = os.path.join(MODEL_DIR, f"{PREFIX}_user_merchant_stats.joblib")


# ============================================================================
# RULE INTERPRETER (Loads declarative rules from JSON config)
# ============================================================================
# This replaces hardcoded lambdas with a generic interpreter that reads
# rule conditions from the JSON config file saved by trx2.py.
# When rules change in trx2.py, they are automatically picked up here.
# ============================================================================

RULES_CONFIG_FILE = os.path.join(MODEL_DIR, "ultimate_fraud_system_fast.joblib_rules.json")


class RuleInterpreter:
    """
    Generic rule interpreter that evaluates declarative rules from JSON config.
    
    Supports operators:
    - Comparison: >, >=, <, <=, ==, !=
    - Range: between (inclusive)
    - Membership: in
    - Compound: and, or
    - Special: field_ne (compare two fields)
    """
    
    def __init__(self, config_path=None):
        self.rule_weights = {}
        self.rule_conditions = {}
        
        # Try to load from config file
        config_path = config_path or RULES_CONFIG_FILE
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                config = json.load(f)
            self.rule_weights = config.get('rule_weights', {})
            self.rule_conditions = config.get('rule_conditions', {})
            print(f"   ✅ Loaded {len(self.rule_conditions)} rules from config")
        else:
            print(f"   ⚠️ Rules config not found, using fallback")
            self._load_fallback_rules()
        
        # ALWAYS inject new rules (in case config is stale from old training)
        # These will be properly exported after next retrain via trx2.py
        self._inject_new_rules()
    
    def _load_fallback_rules(self):
        """Fallback rules if config file doesn't exist"""
        self.rule_weights = {
            'velocity_attack': 30, 'amount_anomaly_extreme': 35, 'high_amount': 25,
            'night_transaction': 10, 'new_device': 15, 'new_country': 20
        }
        self.rule_conditions = {
            'velocity_attack': {'field': 'txns_last_10min', 'op': 'between', 'value': [5, 10]},
            'amount_anomaly_extreme': {'field': 'amount_vs_user_avg', 'op': '>', 'value': 10},
            'high_amount': {'field': 'amount_vs_user_avg', 'op': '>', 'value': 5},
            'night_transaction': {'field': 'is_night', 'op': '==', 'value': 1},
            'new_device': {'field': 'is_new_device', 'op': '==', 'value': 1},
            'new_country': {'field': 'is_new_country', 'op': '==', 'value': 1},
        }
    
    def _inject_new_rules(self):
        """Inject new rules that may not be in saved config (added after training)"""
        # Repeat Fraud Offender - user with 2+ past frauds
        if 'repeat_fraud_offender' not in self.rule_conditions:
            self.rule_weights['repeat_fraud_offender'] = 85
            self.rule_conditions['repeat_fraud_offender'] = {
                'field': 'user_past_fraud_count', 'op': '>=', 'value': 2
            }
        
        # Micro Transaction Velocity - 3+ micro txns in 5 min (HARD BLOCK - card testing)
        if 'micro_txn_velocity' not in self.rule_conditions:
            self.rule_weights['micro_txn_velocity'] = 85
            self.rule_conditions['micro_txn_velocity'] = {
                'field': 'micro_txn_count_5min', 'op': '>=', 'value': 3
            }
        
        # Update fraud_history_high weight to be higher (was 25, now 65)
        if 'fraud_history_high' in self.rule_weights:
            self.rule_weights['fraud_history_high'] = 65
    
    def _evaluate_condition(self, txn, condition):
        """Evaluate a single condition against a transaction dict"""
        op = condition.get('op', '==')
        
        # Compound conditions (and/or)
        if op == 'and':
            return all(self._evaluate_condition(txn, c) for c in condition.get('conditions', []))
        if op == 'or':
            return any(self._evaluate_condition(txn, c) for c in condition.get('conditions', []))
        
        # Special: compare two fields
        if op == 'field_ne':
            fields = condition.get('value', [])
            if len(fields) == 2:
                v1 = str(txn.get(fields[0], '')).upper()
                v2 = str(txn.get(fields[1], '')).upper()
                return v1 != '' and v2 != '' and v1 != v2
            return False
        
        # Get field value
        field = condition.get('field', '')
        value = txn.get(field, 0)
        target = condition.get('value')
        
        # Handle string comparisons (convert to uppercase for 'in' operator)
        if op == 'in':
            if isinstance(value, str):
                return value.upper() in [str(v).upper() for v in target]
            return value in target
        
        # Numeric comparisons
        try:
            value = float(value) if not isinstance(value, str) else 0
            
            if op == '>': return value > target
            if op == '>=': return value >= target
            if op == '<': return value < target
            if op == '<=': return value <= target
            if op == '==': return value == target
            if op == '!=': return value != target
            if op == 'between':
                low, high = target[0], target[1]
                return low <= value <= high
        except (ValueError, TypeError):
            pass
        
        return False
    
    def evaluate(self, txn_dict):
        """Evaluate all rules, return max score and triggered rules"""
        triggered = []
        max_score = 0
        
        for rule_name, condition in self.rule_conditions.items():
            try:
                if self._evaluate_condition(txn_dict, condition):
                    weight = self.rule_weights.get(rule_name, 0)
                    triggered.append((rule_name, weight))
                    max_score = max(max_score, weight)
            except Exception:
                pass
        
        return max_score, triggered
    
    def get_rule_signals(self, txn_dict):
        """Get binary signals for all rules (for BNN input)"""
        signals = []
        for rule_name in self.rule_conditions.keys():
            try:
                condition = self.rule_conditions[rule_name]
                signals.append(1.0 if self._evaluate_condition(txn_dict, condition) else 0.0)
            except:
                signals.append(0.0)
        return signals
    
    def get_rule_signals_ordered(self, txn_dict, rule_names):
        """Get binary signals in EXACT order specified by rule_names.
        
        This is used for BNN inference to ensure signals match training order.
        """
        signals = []
        for rule_name in rule_names:
            try:
                condition = self.rule_conditions.get(rule_name, {})
                signals.append(1.0 if self._evaluate_condition(txn_dict, condition) else 0.0)
            except:
                signals.append(0.0)
        return signals


# Legacy alias for compatibility
class RuleEngine(RuleInterpreter):
    """Alias for backward compatibility"""
    def __init__(self, rule_specs=None, config_path=None):
        super().__init__(config_path)
        # If rule_specs provided (old format), merge with config
        if rule_specs:
            for rule_name, weight in rule_specs:
                self.rule_weights[rule_name] = weight



# ============================================================================
# BNN INFERENCE
# ============================================================================
def predict_bnn(txn_dict, rule_engine, bnn_model, bnn_threshold, bnn_rule_names=None):
    """Predict using BNN with rule signals from RuleInterpreter
    
    If bnn_rule_names is provided, signals will be generated in that exact order
    to match the order used during BNN training.
    """
    if bnn_model is None:
        return {'is_high_risk': False, 'probability': 0.0}
    try:
        # Get rule signals - use saved order if available, otherwise use dict order
        if bnn_rule_names:
            signals = np.array([rule_engine.get_rule_signals_ordered(txn_dict, bnn_rule_names)]).reshape(1, -1)
        else:
            signals = np.array([rule_engine.get_rule_signals(txn_dict)]).reshape(1, -1)
        prob = bnn_model.predict_proba(signals)[0, 1]
        return {'is_high_risk': prob >= bnn_threshold, 'probability': float(prob)}
    except Exception as e:
        return {'is_high_risk': False, 'probability': 0.0}


# ============================================================================
# LOAD SYSTEM
# ============================================================================
def load_system():
    print("=" * 70)
    print("🔮 LOADING CASCADED FRAUD DETECTION SYSTEM")
    print("=" * 70)
    
    system = {}
    
    # ===== Stage 1: BNN =====
    print("\n📦 Loading Stage 1 (Rules + BNN)...")
    if os.path.exists(STAGE1_FILE):
        stage1 = joblib.load(STAGE1_FILE)
        system['bnn_model'] = stage1.get('bnn_model')
        system['bnn_threshold'] = stage1.get('bnn_threshold', 0.5)
        system['hard_block_threshold'] = stage1.get('hard_block_threshold', 85) / 100.0
        system['bnn_rule_names'] = stage1.get('bnn_rule_names', [])  # Rule order for BNN
        print(f"   ✅ BNN loaded (threshold={system['bnn_threshold']:.3f})")
        if system['bnn_rule_names']:
            print(f"   ✅ BNN rule order: {len(system['bnn_rule_names'])} rules")
    else:
        print(f"   ⚠️ Stage 1 not found")
        system['bnn_model'] = None
        system['bnn_threshold'] = 0.5
        system['hard_block_threshold'] = 0.85
    
    # ===== Stage 2: ML Ensemble + Scalers =====
    print("\n📦 Loading Stage 2 (ML Ensemble + Scalers)...")
    if os.path.exists(STAGE2_FILE):
        stage2 = joblib.load(STAGE2_FILE)
        system['models'] = stage2.get('models', {})
        system['optimal_weights'] = stage2.get('optimal_weights', {})
        system['optimal_threshold'] = stage2.get('optimal_threshold', 0.390)
        system['scaler'] = stage2.get('scaler')
        system['quantile_transformer'] = stage2.get('quantile_transformer')
        system['feature_names'] = stage2.get('feature_names', [])
        system['rule_specs'] = stage2.get('rule_specs', [])
        print(f"   ✅ {len(system['models'])} models, {len(system['feature_names'])} features")
        print(f"   ✅ Scalers: RobustScaler + QuantileTransformer")
    else:
        print(f"   ❌ Stage 2 not found")
        return None
    
    # ===== Feature Engineer =====
    print("\n📦 Loading Feature Engineering Config...")
    if os.path.exists(FEATURE_CONFIG_FILE) and os.path.exists(USER_STATS_FILE):
        system['feature_engineer'] = FeatureEngineer(FEATURE_CONFIG_FILE, USER_STATS_FILE)
        print(f"   ✅ FeatureEngineer loaded")
    else:
        print(f"   ⚠️ Feature config not found - using basic features")
        system['feature_engineer'] = None
    
    # Rule Engine - Now loads from config file automatically!
    print("\n📦 Loading Rule Engine from config...")
    system['rule_engine'] = RuleEngine(config_path=RULES_CONFIG_FILE)
    print(f"   ✅ Rule Engine ({len(system['rule_engine'].rule_conditions)} rules)")
    
    return system


# ============================================================================
# INFERENCE
# ============================================================================
def predict(raw_transaction, system, verbose=True):
    """
    Full cascaded prediction with proper feature engineering.
    
    raw_transaction: dict with raw transaction fields (amount, timestamp, user_id, etc.)
    verbose: if True, print step-by-step pipeline progress
    """
    result = {'raw_input': raw_transaction}
    
    # ===== STEP 1: Feature Engineering =====
    if verbose:
        print(f"\n   📥 STEP 1: Raw Input")
        print(f"      └─ Received {len(raw_transaction)} raw columns")
        print(f"      └─ Amount: ${raw_transaction.get('amount', 0):,.2f}")
        print(f"      └─ User: {raw_transaction.get('user_id', 'N/A')}")
    
    if system['feature_engineer']:
        # Create DataFrame with required columns
        df = pd.DataFrame([raw_transaction])
        df_engineered, feature_cols = system['feature_engineer'].engineer_features(df)
        full_features = df_engineered[system['feature_names']].iloc[0].to_dict()
        
        # IMPORTANT: Merge raw transaction fields for rule engine
        # Some rule fields (txns_last_10min, is_burst_txn) may not be in training features
        # but need to be accessible by the rule engine
        for key, value in raw_transaction.items():
            if key not in full_features:
                full_features[key] = value
        
        if verbose:
            print(f"\n   🔧 STEP 2: Feature Engineering")
            print(f"      └─ Input columns: {len(raw_transaction)}")
            print(f"      └─ Engineered features: {len(system['feature_names'])}")
            print(f"      └─ Config: feature_config.json ✓")
            print(f"      └─ User/Merchant stats loaded ✓")
    else:
        # Fallback: fill missing with 0
        full_features = {feat: raw_transaction.get(feat, 0) for feat in system['feature_names']}
        if verbose:
            print(f"\n   🔧 STEP 2: Feature Engineering (fallback mode)")
            print(f"      └─ Using raw features with defaults")
    
    # ===== STEP 2: Rule Engine =====
    if system['rule_engine']:
        rule_score, triggered = system['rule_engine'].evaluate(full_features)
        rule_score_norm = rule_score / 100.0
        result['rule_score'] = rule_score
        result['triggered_rules'] = [r[0] for r in triggered]
        
        if verbose:
            print(f"\n   🚨 STEP 3: Rule Engine (Stage 1)")
            print(f"      └─ Rules evaluated: {len(system['rule_engine'].rule_conditions)}")
            print(f"      └─ Rules triggered: {len(triggered)}")
            print(f"      └─ Score: {rule_score}% (threshold: {int(system['hard_block_threshold']*100)}%)")
            if triggered:
                print(f"      └─ Matched: {', '.join([r[0] for r in triggered[:3]])}")
        
        if rule_score_norm >= system['hard_block_threshold']:
            result['decision'] = 'FRAUD (HARD BLOCK)'
            result['final_score'] = 1.0
            if verbose:
                print(f"\n   🛑 HARD BLOCK TRIGGERED - Skipping ML")
            return result
    else:
        rule_score, rule_score_norm = 0, 0
        result['rule_score'] = 0
        result['triggered_rules'] = []
    
    # ===== STEP 3: BNN High-Risk =====
    if system['bnn_model'] and system['rule_engine']:
        bnn_result = predict_bnn(full_features, system['rule_engine'],
                                  system['bnn_model'], system['bnn_threshold'],
                                  system.get('bnn_rule_names'))
        is_high_risk = bnn_result['is_high_risk']
        result['bnn_probability'] = bnn_result['probability']
        
        if verbose:
            print(f"\n   🧠 STEP 4: BNN High-Risk Identifier")
            print(f"      └─ Rule signals extracted: {len(system['rule_engine'].rule_conditions)}")
            print(f"      └─ Probability: {bnn_result['probability']:.3f}")
            print(f"      └─ Threshold: {system['bnn_threshold']:.3f}")
            print(f"      └─ Classification: {'HIGH RISK ⚠️' if is_high_risk else 'NORMAL ✓'}")
    else:
        is_high_risk = rule_score_norm >= 0.60
    result['is_high_risk'] = is_high_risk
    
    # ===== STEP 4: Scaling =====
    X = pd.DataFrame([[full_features[f] for f in system['feature_names']]], 
                      columns=system['feature_names'])
    
    if verbose:
        print(f"\n   📏 STEP 5: Hybrid Scaling")
    
    # Apply scalers
    if system['scaler']:
        X_scaled = system['scaler'].transform(X)
        if verbose:
            print(f"      └─ RobustScaler applied ✓")
        if system['quantile_transformer']:
            X_scaled = system['quantile_transformer'].transform(X_scaled)
            if verbose:
                print(f"      └─ QuantileTransformer applied ✓")
    else:
        X_scaled = X.values
        if verbose:
            print(f"      └─ No scalers (raw features)")
    
    if verbose:
        print(f"      └─ Output shape: {X_scaled.shape}")
    
    # ===== STEP 5: ML Ensemble =====
    # NOTE: Using AVERAGE strategy (equal weights) as determined optimal by training pipeline
    # The training pipeline found 'average' achieved best F2-score (73.20%) with Precision ≥ 60%
    if verbose:
        print(f"\n   🤖 STEP 6: ML Ensemble Prediction (AVERAGE strategy)")
    
    ml_pred = 0.0
    model_preds = {}
    n_models = len(system['models'])
    equal_weight = 1.0 / n_models  # Average strategy: equal weights for all models
    
    for name, model in system['models'].items():
        try:
            pred = model.predict_proba(X_scaled)[0, 1]
            model_preds[name] = pred
            ml_pred += equal_weight * pred
            if verbose:
                print(f"      └─ {name}: {pred*100:.1f}% × {equal_weight*100:.1f}% = {equal_weight * pred * 100:.1f}%")
        except Exception as e:
            if verbose:
                print(f"      └─ {name}: ERROR - {e}")
    
    result['ml_prediction'] = ml_pred
    result['model_predictions'] = model_preds
    
    if verbose:
        print(f"      └─ ML Fraud Probability: {ml_pred*100:.1f}%")
    
    # ===== STEP 6: Blending =====
    # FIX: Use HIGH RISK blend if EITHER:
    #   1. BNN classifies as HIGH RISK, OR
    #   2. Rule score >= 60% (gray zone with strong rule signals)
    # This ensures strong rule signals (like velocity attacks) are not diluted by BNN
    use_high_risk_blend = is_high_risk or rule_score >= 60  # rule_score is 0-100 scale
    
    if use_high_risk_blend:
        final_score = ml_pred * 0.65 + rule_score_norm * 0.35
        if is_high_risk and rule_score >= 60:
            result['blend_type'] = 'High Risk (BNN + Rules)'
        elif is_high_risk:
            result['blend_type'] = 'High Risk (BNN)'
        else:
            result['blend_type'] = 'High Risk (Rules >= 60%)'
        blend_formula = f"({ml_pred*100:.1f}% × 65%) + ({rule_score_norm*100:.1f}% × 35%)"
    else:
        final_score = ml_pred * 0.90 + rule_score_norm * 0.10
        result['blend_type'] = 'Low Risk'
        blend_formula = f"({ml_pred*100:.1f}% × 90%) + ({rule_score_norm*100:.1f}% × 10%)"
    
    result['final_score'] = final_score
    result['threshold'] = system['optimal_threshold']
    result['decision'] = 'FRAUD' if final_score >= system['optimal_threshold'] else 'LEGITIMATE'
    
    # Determine risk level for interpretability
    # Risk Level Thresholds:
    # - LOW: 0-25% (safe, minimal monitoring)
    # - MEDIUM: 25-36% (standard monitoring, may review)
    # - HIGH: 36%+ (fraud detected, block)
    if final_score >= 0.36:
        risk_level = "HIGH"
    elif final_score >= 0.25:
        risk_level = "MEDIUM"
    else:
        risk_level = "LOW"
    result['risk_level'] = risk_level
    
    if verbose:
        print(f"\n   ⚡ STEP 7: Cascaded Blending")
        print(f"      └─ Mode: {result['blend_type']} (BNN classification)")
        print(f"      └─ Formula: {blend_formula}")
        print(f"      └─ Final Fraud Probability: {final_score*100:.1f}%")
    
    if verbose:
        # Create visual progress bar
        bar_length = 20
        filled = int(final_score * bar_length)
        bar = "█" * filled + "░" * (bar_length - filled)
        
        threshold_pct = system['optimal_threshold'] * 100
        final_pct = final_score * 100
        
        print(f"\n   🎯 FINAL DECISION")
        print(f"      ┌{'─' * 40}┐")
        print(f"      │ Fraud Probability: {final_pct:5.1f}%           │")
        print(f"      │ Decision Threshold: {threshold_pct:5.1f}%          │")
        print(f"      │ Risk Level: {risk_level:8}               │")
        print(f"      │ [{bar}] {final_pct:.0f}%     │")
        print(f"      └{'─' * 40}┘")
        
        if result['decision'] == 'FRAUD' or result['decision'] == 'FRAUD (HARD BLOCK)':
            print(f"\n      🚨 FRAUD DETECTED....TRANSACTION BLOCKED")
        elif risk_level == 'MEDIUM':
            print(f"\n      ⚠️  ELEVATED RISK DETECTED")
            print(f"         Transaction has been flagged for additional verification.")
            print(f"         Alert dispatched to Manual Review Team for final decision.")
            result['manual_review'] = True
        else:
            print(f"\n      ✅ VERDICT: LEGITIMATE ({final_pct:.1f}% < {threshold_pct:.1f}%)")
    
    return result


# ============================================================================
# TEST CASES - Complete 37-column dataset schema
# Matches RealisticFraudDatasetGenerator columns exactly
# ============================================================================

# Current timestamp for test transactions
NOW = datetime.now()

TEST_CASES = [
    # ===== TEST 1: OBVIOUS FRAUD - Stolen Card from High-Risk Country =====
    {"name": "🚨 STOLEN CARD", 
     "description": "High amount from new device, high-risk country (NG)",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_001",
         "user_id": "USER_SUSPECT_001",
         "timestamp": NOW,
         "amount": 2500,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "NG",  # Nigeria - high risk
         "billing_city": "Lagos",
         "shipping_country": "US",
         "shipping_city": "New York",
         "latitude": 6.5244,
         "longitude": 3.3792,
         # === Device Fields ===
         "ip_address": "197.210.45.123",
         "device_location_lat": 6.5244,
         "device_location_lon": 3.3792,
         "device_id": "DEV_NEW_001",
         "device_type": "mobile",
         "device_os": "Android",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Linux; Android 11)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_GIFT_001",
         "merchant_name": "GiftCardsNG",
         "merchant_category": "Gift Cards",
         "merchant_country": "NG",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=5),  # New user
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "pending",
         "user_risk_flag": 1,
         "user_total_txn_count": 2,
         "user_total_txn_amount": 100,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.5,
         "failed_login_attempts_last_24h": 3,
     }},
    
    # ===== TEST 2: VELOCITY ATTACK - Multiple rapid transactions =====
    {"name": "🚨 VELOCITY ATTACK", 
     "description": "Rapid transactions in burst (5 in 10 min)",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_002",
         "user_id": "USER_VELOCITY_001",
         "timestamp": NOW,
         "amount": 450,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "US",
         "billing_city": "Miami",
         "shipping_country": "US",
         "shipping_city": "Miami",
         "latitude": 25.7617,
         "longitude": -80.1918,
         # === Device Fields ===
         "ip_address": "72.45.123.89",
         "device_location_lat": 25.7617,
         "device_location_lon": -80.1918,
         "device_id": "DEV_BURST_001",
         "device_type": "desktop",
         "device_os": "Windows",
         "browser": "Firefox",
         "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_CRYPTO_001",
         "merchant_name": "CryptoExchange",
         "merchant_category": "Crypto",
         "merchant_country": "US",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=90),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 50,
         "user_total_txn_amount": 5000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.003,  # ~4 minutes ago (velocity attack)
         "failed_login_attempts_last_24h": 0,
         # VELOCITY ATTACK indicators - explicit fields for rule engine
         "txns_last_10min": 6,  # 6 transactions in last 10 minutes (triggers velocity_attack rule)
         "is_burst_txn": 1,     # Mark as burst transaction
     }},
    
    # ===== TEST 3: CARD TESTING PATTERN - Micro txns followed by large =====
    {"name": "🚨 CARD TESTING", 
     "description": "Small $3 test, now $800 purchase",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_003",
         "user_id": "USER_CARDTEST_001",
         "timestamp": NOW,
         "amount": 800,  # Large amount after testing
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "US",
         "billing_city": "Chicago",
         "shipping_country": "US",
         "shipping_city": "Chicago",
         "latitude": 41.8781,
         "longitude": -87.6298,
         # === Device Fields ===
         "ip_address": "98.210.55.42",
         "device_location_lat": 41.8781,
         "device_location_lon": -87.6298,
         "device_id": "DEV_TEST_001",
         "device_type": "mobile",
         "device_os": "iOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_TECH_001",
         "merchant_name": "TechGadgets",
         "merchant_category": "Technology",
         "merchant_country": "US",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=3),  # Very new account
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "partial",
         "user_risk_flag": 0,
         "user_total_txn_count": 3,  # Only 3 previous micro transactions
         "user_total_txn_amount": 9,  # $3 + $3 + $3 = $9 (micro amounts)
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.01,
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 4: SUSPICIOUS - Night + Failed KYC + Foreign Merchant =====
    {"name": "⚠️ SUSPICIOUS", 
     "description": "2AM txn, failed KYC, geo mismatch",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_004",
         "user_id": "USER_SUSPECT_002",
         "timestamp": datetime(2024, 12, 20, 2, 30, 0),  # 2:30 AM
         "amount": 600,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "RU",  # Russia - mismatch
         "billing_city": "Moscow",
         "shipping_country": "RU",
         "shipping_city": "Moscow",
         "latitude": 55.7558,
         "longitude": 37.6173,
         # === Device Fields ===
         "ip_address": "95.84.192.45",
         "device_location_lat": 55.7558,
         "device_location_lon": 37.6173,
         "device_id": "DEV_RU_001",
         "device_type": "desktop",
         "device_os": "Windows",
         "browser": "Edge",
         "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Edge/119.0",
         # === Merchant Fields ===
         "merchant_id": "MERCH_TRANSFER_001",
         "merchant_name": "MoneyTransferRU",
         "merchant_category": "Money Transfer",
         "merchant_country": "RU",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=60),
         "user_country": "US",  # User is from US
         "user_segment": "retail",
         "kyc_status": "failed",  # Failed KYC
         "user_risk_flag": 1,
         "user_total_txn_count": 10,
         "user_total_txn_amount": 1500,
         "user_past_fraud_count": 1,  # Has fraud history
         "days_since_last_txn": 5,
         "failed_login_attempts_last_24h": 4,
     }},
    
    # ===== TEST 5: NORMAL - Trusted user, regular purchase =====
    {"name": "✅ NORMAL", 
     "description": "Established user, small daytime purchase",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_005",
         "user_id": "USER_TRUSTED_001",
         "timestamp": datetime(2024, 12, 20, 14, 30, 0),  # 2:30 PM
         "amount": 45,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "POS",
         # === Location Fields ===
         "billing_country": "US",
         "billing_city": "San Francisco",
         "shipping_country": "US",
         "shipping_city": "San Francisco",
         "latitude": 37.7749,
         "longitude": -122.4194,
         # === Device Fields ===
         "ip_address": "50.193.45.78",
         "device_location_lat": 37.7749,
         "device_location_lon": -122.4194,
         "device_id": "DEV_TRUSTED_001",
         "device_type": "mobile",
         "device_os": "iOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_FOOD_001",
         "merchant_name": "LocalCafe",
         "merchant_category": "Food",
         "merchant_country": "US",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=730),  # 2 years old
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 500,
         "user_total_txn_amount": 25000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 2,
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 6: NORMAL - High-value trusted customer =====
    {"name": "✅ TRUSTED HIGH", 
     "description": "Luxury spender, consistent pattern",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_006",
         "user_id": "USER_VIP_001",
         "timestamp": datetime(2024, 12, 20, 11, 0, 0),  # 11 AM
         "amount": 1200,  # High but within user's normal range
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "US",
         "billing_city": "Beverly Hills",
         "shipping_country": "US",
         "shipping_city": "Beverly Hills",
         "latitude": 34.0736,
         "longitude": -118.4004,
         # === Device Fields ===
         "ip_address": "76.127.89.45",
         "device_location_lat": 34.0736,
         "device_location_lon": -118.4004,
         "device_id": "DEV_VIP_001",
         "device_type": "desktop",
         "device_os": "macOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_0)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_TRAVEL_001",
         "merchant_name": "LuxuryTravel",
         "merchant_category": "Travel",
         "merchant_country": "US",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=1095),  # 3 years old
         "user_country": "US",
         "user_segment": "corporate",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 1200,
         "user_total_txn_amount": 480000,  # Avg $400/txn - luxury spender
         "user_past_fraud_count": 0,
         "days_since_last_txn": 3,
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 7: IMPOSSIBLE TRAVEL - User in 2 locations too fast =====
    {"name": "🚨 IMPOSSIBLE TRAVEL", 
     "description": "NYC 2 hours ago, now in London (speed > 900 km/h)",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_007",
         "user_id": "USER_TRAVEL_001",
         "timestamp": NOW,
         "amount": 1500,
         "currency": "GBP",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "GB",
         "billing_city": "London",
         "shipping_country": "GB",
         "shipping_city": "London",
         "latitude": 51.5074,
         "longitude": -0.1278,
         # === Device Fields ===
         "ip_address": "82.132.45.67",
         "device_location_lat": 51.5074,
         "device_location_lon": -0.1278,
         "device_id": "DEV_TRAVEL_001",
         "device_type": "mobile",
         "device_os": "Android",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Linux; Android 13)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_RETAIL_UK",
         "merchant_name": "Harrods",
         "merchant_category": "Retail",
         "merchant_country": "GB",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=365),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 100,
         "user_total_txn_amount": 15000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.08,  # ~2 hours - impossible travel from NYC
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 8: ACCOUNT TAKEOVER - New device + failed logins + night =====
    {"name": "🚨 ACCOUNT TAKEOVER", 
     "description": "Multiple failed logins, new device, 3AM purchase",
     "transaction": {
         # === Core Transaction Fields ===
         "transaction_id": "TXN_TEST_008",
         "user_id": "USER_ATO_001",
         "timestamp": datetime(2024, 12, 20, 3, 15, 0),  # 3:15 AM
         "amount": 2000,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         # === Location Fields ===
         "billing_country": "BR",
         "billing_city": "São Paulo",
         "shipping_country": "BR",
         "shipping_city": "São Paulo",
         "latitude": -23.5505,
         "longitude": -46.6333,
         # === Device Fields ===
         "ip_address": "177.92.45.123",
         "device_location_lat": -23.5505,
         "device_location_lon": -46.6333,
         "device_id": "DEV_NEW_ATO_001",
         "device_type": "desktop",
         "device_os": "Linux",
         "browser": "Brave",
         "user_agent": "Mozilla/5.0 (X11; Linux x86_64)",
         # === Merchant Fields ===
         "merchant_id": "MERCH_ELECTRONICS",
         "merchant_name": "ElectronicsBR",
         "merchant_category": "Technology",
         "merchant_country": "BR",
         "payment_method": "card",
         # === User Fields ===
         "user_signup_date": NOW - timedelta(days=500),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 200,
         "user_total_txn_amount": 20000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 15,
         "failed_login_attempts_last_24h": 5,  # Multiple failed logins
     }},
    
    # ===== TEST 9: DEVICE FINGERPRINT CHAOS - Multiple devices/IPs =====
    {"name": "🚨 DEVICE CHAOS", 
     "description": "5+ devices and 5+ IPs in 24 hours",
     "transaction": {
         "transaction_id": "TXN_TEST_009",
         "user_id": "USER_CHAOS_001",
         "timestamp": NOW,
         "amount": 750,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "US",
         "billing_city": "Denver",
         "shipping_country": "US",
         "shipping_city": "Denver",
         "latitude": 39.7392,
         "longitude": -104.9903,
         "ip_address": "192.168.50.99",
         "device_location_lat": 39.7392,
         "device_location_lon": -104.9903,
         "device_id": "DEV_CHAOS_009",
         "device_type": "tablet",
         "device_os": "Android",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Linux; Android 12)",
         "merchant_id": "MERCH_GAMING_001",
         "merchant_name": "GameStore",
         "merchant_category": "Gaming",
         "merchant_country": "US",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=30),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 20,
         "user_total_txn_amount": 2000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 1,
         "failed_login_attempts_last_24h": 0,
         # Device chaos indicators
         "distinct_devices_24hr": 7,
         "distinct_ips_24hr": 6,
     }},
    
    # ===== TEST 10: NEW COUNTRY HIGH AMOUNT =====
    {"name": "🚨 NEW COUNTRY HIGH", 
     "description": "First transaction from new country, high amount",
     "transaction": {
         "transaction_id": "TXN_TEST_010",
         "user_id": "USER_NEWCOUNTRY_001",
         "timestamp": NOW,
         "amount": 1800,
         "currency": "EUR",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "DE",
         "billing_city": "Berlin",
         "shipping_country": "DE",
         "shipping_city": "Berlin",
         "latitude": 52.5200,
         "longitude": 13.4050,
         "ip_address": "91.64.123.45",
         "device_location_lat": 52.5200,
         "device_location_lon": 13.4050,
         "device_id": "DEV_NEWCOUNTRY_001",
         "device_type": "mobile",
         "device_os": "iOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0)",
         "merchant_id": "MERCH_LUXURY_DE",
         "merchant_name": "LuxuryBerlin",
         "merchant_category": "Luxury",
         "merchant_country": "DE",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=200),
         "user_country": "US",  # User is from US, billing now from DE
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 80,
         "user_total_txn_amount": 8000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 3,
         "failed_login_attempts_last_24h": 0,
         # New country indicator
         "is_new_country": 1,
     }},
    
    # ===== TEST 11: REPEAT FRAUD OFFENDER =====
    {"name": "🚨 REPEAT OFFENDER", 
     "description": "User with past fraud history making high-value purchase",
     "transaction": {
         "transaction_id": "TXN_TEST_011",
         "user_id": "USER_REPEAT_001",
         "timestamp": NOW,
         "amount": 500,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "US",
         "billing_city": "Atlanta",
         "shipping_country": "US",
         "shipping_city": "Atlanta",
         "latitude": 33.7490,
         "longitude": -84.3880,
         "ip_address": "68.45.123.78",
         "device_location_lat": 33.7490,
         "device_location_lon": -84.3880,
         "device_id": "DEV_REPEAT_001",
         "device_type": "desktop",
         "device_os": "Windows",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
         "merchant_id": "MERCH_ELEC_001",
         "merchant_name": "ElectronicsUSA",
         "merchant_category": "Electronics",
         "merchant_country": "US",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=400),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 1,
         "user_total_txn_count": 150,
         "user_total_txn_amount": 15000,
         "user_past_fraud_count": 3,  # Has 3 past frauds!
         "days_since_last_txn": 7,
         "failed_login_attempts_last_24h": 0,
     }},
    
    # ===== TEST 12: MICRO TRANSACTION PATTERN =====
    {"name": "⚠️ MICRO TXN PATTERN", 
     "description": "Multiple $1-$5 transactions in quick succession",
     "transaction": {
         "transaction_id": "TXN_TEST_012",
         "user_id": "USER_MICRO_001",
         "timestamp": NOW,
         "amount": 2.50,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "US",
         "billing_city": "Portland",
         "shipping_country": "US",
         "shipping_city": "Portland",
         "latitude": 45.5152,
         "longitude": -122.6784,
         "ip_address": "73.42.156.89",
         "device_location_lat": 45.5152,
         "device_location_lon": -122.6784,
         "device_id": "DEV_MICRO_001",
         "device_type": "mobile",
         "device_os": "Android",
         "browser": "Chrome",
         "user_agent": "Mozilla/5.0 (Linux; Android 13)",
         "merchant_id": "MERCH_DONATION_001",
         "merchant_name": "CharityDonate",
         "merchant_category": "Nonprofit",
         "merchant_country": "US",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=10),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "pending",
         "user_risk_flag": 0,
         "user_total_txn_count": 8,
         "user_total_txn_amount": 24,  # 8 transactions of $3 average
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.001,  # Very recent
         "failed_login_attempts_last_24h": 0,
         # Micro transaction indicators
         "micro_txn_count_5min": 4,
         "time_since_last_micro_txn_min": 1,
     }},
    
    # ===== TEST 13: SPEED OF LIGHT VIOLATION =====
    {"name": "🚨 TELEPORTATION", 
     "description": "Physically impossible travel (>1500 km/h)",
     "transaction": {
         "transaction_id": "TXN_TEST_013",
         "user_id": "USER_TELEPORT_001",
         "timestamp": NOW,
         "amount": 900,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "JP",
         "billing_city": "Tokyo",
         "shipping_country": "JP",
         "shipping_city": "Tokyo",
         "latitude": 35.6762,
         "longitude": 139.6503,
         "ip_address": "103.5.140.50",
         "device_location_lat": 35.6762,
         "device_location_lon": 139.6503,
         "device_id": "DEV_TELEPORT_001",
         "device_type": "mobile",
         "device_os": "iOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0)",
         "merchant_id": "MERCH_RETAIL_JP",
         "merchant_name": "TokyoShop",
         "merchant_category": "Retail",
         "merchant_country": "JP",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=180),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 60,
         "user_total_txn_amount": 6000,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 0.02,  # ~30 min ago - was in NYC
         "failed_login_attempts_last_24h": 0,
         # Speed of light violation
         "travel_speed_kmh": 2000,  # NYC to Tokyo in 30 min = impossible!
     }},
    
    # ===== TEST 14: GRAY ZONE - Borderline Case =====
    {"name": "⚠️ GRAY ZONE", 
     "description": "Multiple weak signals, borderline decision",
     "transaction": {
         "transaction_id": "TXN_TEST_014",
         "user_id": "USER_GRAY_001",
         "timestamp": datetime(2024, 12, 20, 22, 45, 0),  # Late night 10:45 PM
         "amount": 350,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "e-commerce",
         "billing_country": "US",
         "billing_city": "Seattle",
         "shipping_country": "US",
         "shipping_city": "Seattle",
         "latitude": 47.6062,
         "longitude": -122.3321,
         "ip_address": "67.82.45.123",
         "device_location_lat": 47.6062,
         "device_location_lon": -122.3321,
         "device_id": "DEV_GRAY_001",
         "device_type": "desktop",
         "device_os": "macOS",
         "browser": "Safari",
         "user_agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_0)",
         "merchant_id": "MERCH_ONLINE_001",
         "merchant_name": "OnlineStore",
         "merchant_category": "Retail",
         "merchant_country": "US",
         "payment_method": "card",
         "user_signup_date": NOW - timedelta(days=45),
         "user_country": "US",
         "user_segment": "retail",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 15,
         "user_total_txn_amount": 800,
         "user_past_fraud_count": 0,
         "days_since_last_txn": 4,
         "failed_login_attempts_last_24h": 1,
         # Weak signals
         "is_night": 1,
         "is_new_device": 1,
     }},
    
    # ===== TEST 15: CORPORATE HIGH-VALUE LEGITIMATE =====
    {"name": "✅ CORPORATE LEGIT", 
     "description": "Large corporate purchase, established account",
     "transaction": {
         "transaction_id": "TXN_TEST_015",
         "user_id": "USER_CORP_001",
         "timestamp": datetime(2024, 12, 20, 10, 0, 0),  # 10 AM business hours
         "amount": 15000,
         "currency": "USD",
         "transaction_type": "purchase",
         "channel": "B2B",
         "billing_country": "US",
         "billing_city": "New York",
         "shipping_country": "US",
         "shipping_city": "New York",
         "latitude": 40.7128,
         "longitude": -74.0060,
         "ip_address": "64.233.160.0",  # Corporate IP
         "device_location_lat": 40.7128,
         "device_location_lon": -74.0060,
         "device_id": "DEV_CORP_001",
         "device_type": "desktop",
         "device_os": "Windows",
         "browser": "Edge",
         "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Edge/119.0",
         "merchant_id": "MERCH_B2B_001",
         "merchant_name": "B2BSupplies",
         "merchant_category": "B2B",
         "merchant_country": "US",
         "payment_method": "wire",
         "user_signup_date": NOW - timedelta(days=1500),  # 4+ years
         "user_country": "US",
         "user_segment": "corporate",
         "kyc_status": "verified",
         "user_risk_flag": 0,
         "user_total_txn_count": 2000,
         "user_total_txn_amount": 5000000,  # $5M historical
         "user_past_fraud_count": 0,
         "days_since_last_txn": 1,
         "failed_login_attempts_last_24h": 0,
     }},
]


# ============================================================================
# MAIN
# ============================================================================
def main():
    system = load_system()
    if not system:
        print("\n❌ Failed to load system")
        return
    
    print("\n" + "=" * 70)
    print("🧪 EVALUATING TEST CASES")
    print("=" * 70)
    
    for i, tc in enumerate(TEST_CASES, 1):
        print(f"\n{'━' * 70}")
        print(f"📋 TEST {i}/{len(TEST_CASES)}: {tc['name']}")
        print(f"   {tc['description']}")
        print(f"{'━' * 70}")
        
        # Predict with verbose=True shows all pipeline steps
        result = predict(tc['transaction'], system, verbose=True)
    
    print("\n" + "=" * 70)
    print("✅ ALL TESTS COMPLETE")
    print("=" * 70)


if __name__ == "__main__":
    main()


🔮 LOADING CASCADED FRAUD DETECTION SYSTEM

📦 Loading Stage 1 (Rules + BNN)...
   ✅ BNN loaded (threshold=0.491)
   ✅ BNN rule order: 27 rules

📦 Loading Stage 2 (ML Ensemble + Scalers)...
   ✅ 3 models, 235 features
   ✅ Scalers: RobustScaler + QuantileTransformer

📦 Loading Feature Engineering Config...
   📊 Total training features: 235
✅ Feature config loaded: C:\Users\ADMIN\Documents\TechFiesta_2026\ML_Models_6.0_Latest\ultimate_fraud_system_fast.joblib_feature_config.json
   👤 Users in stats: 8000
   🏪 Merchants in stats: 2000
✅ User/Merchant stats loaded: C:\Users\ADMIN\Documents\TechFiesta_2026\ML_Models_6.0_Latest\ultimate_fraud_system_fast.joblib_user_merchant_stats.joblib
   ✅ FeatureEngineer loaded

📦 Loading Rule Engine from config...
   ✅ Loaded 29 rules from config
   ✅ Rule Engine (29 rules)

🧪 EVALUATING TEST CASES

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📋 TEST 1/15: 🚨 STOLEN CARD
   High amount from new device, high-risk country (NG)
━━━━